In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType

In [3]:

# ====================== INITIALISATION ======================
spark = SparkSession.builder \
    .appName("TrainALSModel") \
    .master("spark://namenode:7077") \
    .config("spark.cassandra.connection.host", "cassandra") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.cores", "2") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# ====================== CHARGEMENT DES DONNÉES ======================
ratings_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", FloatType(), True),
    StructField("timestamp", StringType(), True),
])

movies_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("genres", StringType(), True),
])

print("📥 Lecture des fichiers CSV depuis HDFS...")
ratings_df = spark.read.csv("hdfs:///input/rating.csv", header=True, schema=ratings_schema)
movies_df = spark.read.csv("hdfs:///input/movie.csv", header=True, schema=movies_schema)

📥 Lecture des fichiers CSV depuis HDFS...


In [5]:
# ====================== NETTOYAGE ======================
# Supprimer les lignes avec valeurs nulles
ratings_df = ratings_df.dropna(subset=["userId", "movieId", "rating"])

In [6]:
# ====================== DIVISION TRAIN / TEST ======================
train_df, test_df = ratings_df.randomSplit([0.8, 0.2], seed=42)

In [7]:
# ====================== ENTRAÎNEMENT ======================
print("🤖 Entraînement du modèle ALS...")
als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    nonnegative=True,
    implicitPrefs=False,
    coldStartStrategy="drop",  # pour éviter les NaN en test
    rank=12,
    maxIter=15,
    regParam=0.05
)

model = als.fit(train_df)

🤖 Entraînement du modèle ALS...


25/04/30 11:42:09 WARN HeartbeatReceiver: Removing executor 2 with no recent heartbeats: 152963 ms exceeds timeout 120000 ms
25/04/30 11:42:09 WARN HeartbeatReceiver: Removing executor 5 with no recent heartbeats: 167371 ms exceeds timeout 120000 ms
25/04/30 11:42:09 WARN HeartbeatReceiver: Removing executor 12 with no recent heartbeats: 156997 ms exceeds timeout 120000 ms
25/04/30 11:42:09 WARN HeartbeatReceiver: Removing executor 8 with no recent heartbeats: 167422 ms exceeds timeout 120000 ms
25/04/30 11:42:09 WARN HeartbeatReceiver: Removing executor 7 with no recent heartbeats: 167494 ms exceeds timeout 120000 ms
25/04/30 11:42:09 WARN HeartbeatReceiver: Removing executor 4 with no recent heartbeats: 167857 ms exceeds timeout 120000 ms
25/04/30 11:42:09 WARN HeartbeatReceiver: Removing executor 11 with no recent heartbeats: 156053 ms exceeds timeout 120000 ms
25/04/30 11:42:09 WARN HeartbeatReceiver: Removing executor 6 with no recent heartbeats: 154289 ms exceeds timeout 120000 m

In [8]:
# ====================== ÉVALUATION ======================
print("📊 Évaluation du modèle...")
predictions = model.transform(test_df)

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print(f"✅ RMSE sur l'ensemble test : {rmse:.4f}")

📊 Évaluation du modèle...


[Stage 184:=====================================================> (28 + 1) / 29]

✅ RMSE sur l'ensemble test : 0.7926


In [9]:
# ====================== ENREGISTREMENT ======================
print("💾 Sauvegarde du modèle dans HDFS (/models/als)...")
model.write().overwrite().save("hdfs:///models/als")

print("🎉 Modèle entraîné et sauvegardé avec succès.")

💾 Sauvegarde du modèle dans HDFS (/models/als)...


[Stage 253:=======================================>                (7 + 3) / 10]

🎉 Modèle entraîné et sauvegardé avec succès.
